Guardamos el DataFrame df_final en un archivo Excel llamado 'archivo_transformado.xlsx'.
Mostramos el DataFrame df_final en la pantalla.

In [13]:
import pandas as pd
import os

# Rutas de los archivos (ajusta según tu sistema)
input_path = os.path.join("..", "data", "raw_data", "Poblacion_02.xlsx")
output_path = os.path.join("..", "data", "processed_data", "archivo_transformado.xlsx")

# 1. Leer el archivo Excel
try:
    df = pd.read_excel(input_path)
    print("Archivo leído correctamente.")
except FileNotFoundError:
    print(f"Error: Archivo no encontrado en {input_path}")
    exit()

# 2. Limpieza y filtrado de datos
df = df[df['Grupo quinquenal de edad'] != 'Total']
df = df[df['Entidad federativa'] != 'Estados Unidos Mexicanos']
df = df.drop(columns=['Total.1', 'Total.2', 'Total.3', 'Total.4', 'Total.5', 'Total'], errors='ignore')
print("Datos limpiados y filtrados.")

# 3. Columnas fijas
fixed_columns = ['Entidad federativa', 'Grupo quinquenal de edad']

# 4. Transformación con melt
df_hombres = pd.melt(df, id_vars=fixed_columns, value_vars=[col for col in df.columns if col.startswith('Hombres')], var_name='Género', value_name='Cantidad')
df_hombres['Género'] = 'Hombres'

df_mujeres = pd.melt(df, id_vars=fixed_columns, value_vars=[col for col in df.columns if col.startswith('Mujeres')], var_name='Género', value_name='Cantidad')
df_mujeres['Género'] = 'Mujeres'

df_year = pd.melt(df, id_vars=fixed_columns, value_vars=[col for col in df.columns if col.startswith('Año')], var_name='Año', value_name='Año_valor')
df_year['Año_valor'] = df_year['Año_valor'].astype(str)  # Convertir a string antes de extraer el año
df_year['Año'] = df_year['Año_valor'].astype(int)
df_year = df_year.drop(columns=['Año_valor'])
print("Datos transformados con melt.")

# 5. Combinación de DataFrames
df_hombres['temp_index'] = df_hombres.groupby(fixed_columns).cumcount()
df_mujeres['temp_index'] = df_mujeres.groupby(fixed_columns).cumcount()
df_year['temp_index'] = df_year.groupby(fixed_columns).cumcount()

df_hombres = pd.merge(df_hombres, df_year, on=fixed_columns + ['temp_index'], how='left')
df_mujeres = pd.merge(df_mujeres, df_year, on=fixed_columns + ['temp_index'], how='left')

df_hombres = df_hombres.drop(columns=['temp_index'])
df_mujeres = df_mujeres.drop(columns=['temp_index'])
print("DataFrames combinados.")

# 6. Concatenación y ordenación
df_final = pd.concat([df_hombres, df_mujeres], ignore_index=True)
df_final = df_final[['Entidad federativa', 'Grupo quinquenal de edad', 'Género', 'Año', 'Cantidad']]

# 7. Interpolación lineal
df_final = df_final.set_index(['Entidad federativa', 'Grupo quinquenal de edad', 'Género'])

def interpolar(grupo):
    if 2015 not in grupo['Año'].values:
        año_2010 = grupo[grupo['Año'] == 2010].iloc[0]
        año_2020 = grupo[grupo['Año'] == 2020].iloc[0]

        cantidad_2010 = año_2010['Cantidad']
        cantidad_2020 = año_2020['Cantidad']

        cantidad_2015 = cantidad_2010 + (cantidad_2020 - cantidad_2010) * (2015 - 2010) / (2020 - 2010)

        nueva_fila = año_2010.copy()
        nueva_fila['Año'] = 2015
        nueva_fila['Cantidad'] = int(round(cantidad_2015))

        grupo = pd.concat([grupo, pd.DataFrame([nueva_fila])], ignore_index=True)
        grupo = grupo.sort_values('Año')
    return grupo

df_final = df_final.groupby(level=[0, 1, 2]).apply(interpolar).reset_index(level=[0, 1, 2])
df_final = df_final.reset_index(drop=True)
print("Interpolación lineal realizada.")


# 8. Crear la columna 'cve_ent' (usando valores distintos)

# Obtener los valores distintos de la columna 'Entidad federativa'
entidades_distintas = df_final['Entidad federativa'].unique()

# Crear el diccionario cve_ent_dict automáticamente
cve_ent_dict = {}
for i, entidad in enumerate(entidades_distintas):
    cve_ent_dict[entidad] = f"{i+1:02}"  # Formato con dos dígitos (01, 02, ..., 32)

def asignar_cve_ent(entidad):
    cve = cve_ent_dict.get(entidad)
    if cve is None:
        print(f"Advertencia: Entidad federativa no encontrada en el diccionario: {entidad}")
    return cve

df_final['cve_ent'] = df_final['Entidad federativa'].apply(asignar_cve_ent)

# Convertir cve_ent a entero (importante para la unión con el shapefile)
#df_final['cve_ent'] = df_final['cve_ent'].astype(int)

# 9. Guardar el resultado
try:
    df_final.to_excel(output_path, index=False)
    print(f"Datos guardados en {output_path}")
except Exception as e:
    print(f"Error al guardar el archivo: {e}")

# Mostrar los primeros registros del DataFrame final (opcional)
print("\nPrimeros registros del DataFrame final:")
print(df_final.head())

Archivo leído correctamente.
Datos limpiados y filtrados.
Datos transformados con melt.
DataFrames combinados.
Interpolación lineal realizada.
Datos guardados en ..\data\processed_data\archivo_transformado.xlsx

Primeros registros del DataFrame final:
  Entidad federativa Grupo quinquenal de edad   Género   Año  Cantidad cve_ent
0     Aguascalientes               0 a 4 años  Hombres  1990     49576      01
1     Aguascalientes               0 a 4 años  Hombres  1995     56920      01
2     Aguascalientes               0 a 4 años  Hombres  2000     58831      01
3     Aguascalientes               0 a 4 años  Hombres  2005     60406      01
4     Aguascalientes               0 a 4 años  Hombres  2010     61941      01


In [14]:
print(df_final.head())

  Entidad federativa Grupo quinquenal de edad   Género   Año  Cantidad cve_ent
0     Aguascalientes               0 a 4 años  Hombres  1990     49576      01
1     Aguascalientes               0 a 4 años  Hombres  1995     56920      01
2     Aguascalientes               0 a 4 años  Hombres  2000     58831      01
3     Aguascalientes               0 a 4 años  Hombres  2005     60406      01
4     Aguascalientes               0 a 4 años  Hombres  2010     61941      01


In [ ]:
df_final

,Entidad federativa,Grupo quinquenal de edad,Género,Año,Cantidad,cve_ent
0,Aguascalientes,0 a 4 años,Hombres,1990,49576,01
1,Aguascalientes,0 a 4 años,Hombres,1995,56920,01
2,Aguascalientes,0 a 4 años,Hombres,2000,58831,01
3,Aguascalientes,0 a 4 años,Hombres,2005,60406,01
4,Aguascalientes,0 a 4 años,Hombres,2010,61941,01
...,...,...,...,...,...,...
9851,Zacatecas,No especificado,Mujeres,2000,4250,32
9852,Zacatecas,No especificado,Mujeres,2005,5435,32
9853,Zacatecas,No especificado,Mujeres,2010,6610,32
9854,Zacatecas,No especificado,Mujeres,2015,3776,32


In [15]:
import altair as alt

# ... (código anterior para crear df_final) ...

# 1. Preparar los datos para el heatmap (reshaping)
df_reshaped = df_final.pivot_table(index='Año', columns='Entidad federativa', values='Cantidad', aggfunc='sum').reset_index()

# 2. Convertir 'Año' a string (necesario para Altair en algunos casos)
df_reshaped['Año'] = df_reshaped['Año'].astype(str) # <--- AÑADIDO

df_reshaped = df_reshaped.rename_axis(None, axis=1) # No es necesario renombrar la columna 'Año' a 'year'

df_reshaped = df_reshaped.melt(id_vars=['Año'], var_name='states', value_name='population') # Usar 'Año' (no 'year')

# 3. Crear el heatmap con Altair
alt.themes.enable("dark")

heatmap = alt.Chart(df_reshaped).mark_rect().encode(
    y=alt.Y('Año:O', axis=alt.Axis(title="Year", titleFontSize=16, titlePadding=15, titleFontWeight=900, labelAngle=0)), # Usar 'Año'
    x=alt.X('states:O', axis=alt.Axis(title="States", titleFontSize=16, titlePadding=15, titleFontWeight=900)),
    color=alt.Color('max(population):Q',
                     legend=alt.Legend(title=" "),
                     scale=alt.Scale(scheme="blueorange")),
    stroke=alt.value('black'),
    strokeWidth=alt.value(0.25)
).properties(width=900).configure_axis(
    labelFontSize=12,
    titleFontSize=12
)

heatmap

alt.Chart(...)

In [22]:
print(shapefile.columns)

Index(['CVEGEO', 'CVE_ENT', 'NOMGEO', 'geometry'], dtype='object')


In [23]:
df_final['Entidad federativa'] = df_final['Entidad federativa'].str.strip()  # Remove leading/trailing spaces
df_final['Entidad federativa'] = df_final['Entidad federativa'].str.lower()  # Convert to lowercase (optional)
shapefile['NOMGEO'] = shapefile['NOMGEO'].str.strip()
shapefile['NOMGEO'] = shapefile['NOMGEO'].str.lower()

In [17]:
df_2020 = df_final[df_final['Año'] == 2020].copy()
df_2020['cve_ent'] = df_2020['cve_ent'].astype(int)

# 4. Agrupar por entidad federativa y sumar cantidades
df_2020_agrupado = df_2020.groupby(['Entidad federativa', 'cve_ent'])['Cantidad'].sum().reset_index()
df_2020_agrupado

,Entidad federativa,cve_ent,Cantidad
0,aguascalientes,1,1425607
1,baja california,2,3769020
2,baja california sur,3,798447
3,campeche,4,928363
4,chiapas,5,5543828
5,chihuahua,6,3741869
6,ciudad de méxico,7,9209944
7,coahuila de zaragoza,8,3146771
8,colima,9,731391
9,durango,10,1832650


In [ ]:
# import geopandas as gpd
# import altair as alt
# import pandas as pd
# import os

# # 1. Cargar y preparar datos del shapefile
# #shapefile_path = "/Volumes/abdel_julio/metro/794551132173_s/mg_2024_integrado/conjunto_de_datos/00ent.shp"

# shapefile_path = (r'E:\\metro\\794551132173_s\\mg_2024_integrado\\conjunto_de_datos\\00ent.shp')
# shapefile = gpd.read_file(shapefile_path)

# # ***IMPORTANT***: Simplify geometry
# shapefile["geometry"] = shapefile.geometry.simplify(tolerance=0.01)
# shapefile = shapefile.set_geometry("geometry")

# shapefile['CVE_ENT'] = shapefile['CVE_ENT'].astype(int)

# # 2. Cargar y preparar datos de población (df_final)
# # ... (Aquí va tu código para cargar o crear df_final) ...

# # # 3. Filtrar df_final para el año 2020 and get cve_ent
# # df_2020 = df_final[df_final['Año'] == 2020].copy()
# # df_2020['cve_ent'] = df_2020['cve_ent'].astype(int)

# # # 4. Agrupar por entidad federativa y sumar cantidades
# # df_2020_agrupado = df_2020.groupby(['Entidad federativa', 'cve_ent'])['Cantidad'].sum().reset_index()

# # 5. Convert shapefile to GeoJSON features (for Altair)
# geojson_features = []
# for _, row in shapefile.iterrows():
#     geojson_features.append({
#         "type": "Feature",
#         "geometry": row.geometry.__geo_interface__,
#         "properties": {
#             "CVE_ENT": row["CVE_ENT"],
#             "NOMGEO": row['NOMGEO']
#         }
#     })

# geojson_data = {"type": "FeatureCollection", "features": geojson_features}

# # 6. Configurar tema
# alt.themes.enable("dark")

# # 7. Mapa coroplético (solo para 2020, población total)
# choropleth = alt.Chart(alt.Data(values=geojson_data["features"])).mark_geoshape(
#     stroke="#154360",
#     strokeWidth=0.5
# ).transform_lookup(
#     lookup='CVE_ENT',  # Key in GeoJSON
#     from_=alt.LookupData(df_2020_agrupado, 'cve_ent', ['Cantidad', 'Entidad federativa']) # Key and data in df
# ).encode(
#     color=alt.Color('Cantidad:Q', scale=alt.Scale(scheme='blues'), title="Población total en 2020"),
#     tooltip=['NOMGEO:N', 'Cantidad:Q']
# ).properties(
#     width=1200,
#     height=900
# ).project(
#     type="conicConformal",
#     center=[-102, 23],
#     parallels=[14, 28],
#     rotate=[97, -23],
#     fit=geojson_data
# )

# choropleth.show()

KeyboardInterrupt: 

In [19]:
import geopandas as gpd
import pandas as pd

# 1. Cargar y preparar el shapefile
#shapefile_path = "/Volumes/abdel_julio/metro/794551132173_s/mg_2024_integrado/conjunto_de_datos/00ent.shp"

shapefile_path = r'E:\\metro\\794551132173_s\\mg_2024_integrado\\conjunto_de_datos\\00ent.shp'
shapefile = gpd.read_file(shapefile_path)

shapefile["geometry"] = shapefile.geometry.simplify(tolerance=0.01)
shapefile = shapefile.set_geometry("geometry")

shapefile['CVE_ENT'] = shapefile['CVE_ENT'].astype(int)

# 2. Cargar y preparar los datos de población (df_final)
# ... (Aquí va tu código para cargar o crear df_final) ...

# 3. Filtrar datos por año
df_2020 = df_final[df_final['Año'] == 2020].copy()
df_2020['cve_ent'] = df_2020['cve_ent'].astype(int)

# 4. Agrupar datos por entidad
df_2020_agrupado = df_2020.groupby(['Entidad federativa', 'cve_ent'])['Cantidad'].sum().reset_index()

# *** 5. Unir shapefile y datos de población (Merge) ***
merged_data = shapefile.merge(df_2020_agrupado, left_on='CVE_ENT', right_on='cve_ent', how='left')

# *** 6. Verificar la unión de datos ***
print("Primeras 10 filas de los datos unidos:")
print(merged_data.head(10))

print("\nInformación sobre los tipos de datos:")
print(merged_data.info())

# *** 7. Contar valores nulos en la columna 'Cantidad' ***
print("\nConteo de valores nulos en la columna 'Cantidad':")
print(merged_data['Cantidad'].isnull().sum())

# *** 8. Mostrar algunas filas donde 'Cantidad' es nulo ***
filas_con_nulos = merged_data[merged_data['Cantidad'].isnull()]
print("\nFilas donde 'Cantidad' es nulo:")
print(filas_con_nulos)

# *** 9. Identificar CVE_ENTs en shapefile sin correspondencia en df_2020_agrupado ***
cve_ent_sin_correspondencia = shapefile[~shapefile['CVE_ENT'].isin(df_2020_agrupado['cve_ent'])]
print("\nCVE_ENTs sin correspondencia en df_2020_agrupado:")
print(cve_ent_sin_correspondencia)

# *** 10. Continuar con la creación del mapa (si la unión es correcta) ***
# ... (El código para crear el mapa con Altair va aquí) ...

Primeras 10 filas de los datos unidos:
  CVEGEO  CVE_ENT                NOMGEO  \
0     01        1        Aguascalientes   
1     02        2       Baja California   
2     03        3   Baja California Sur   
3     04        4              Campeche   
4     05        5  Coahuila de Zaragoza   
5     06        6                Colima   
6     07        7               Chiapas   
7     08        8             Chihuahua   
8     09        9      Ciudad de México   
9     10       10               Durango   

                                            geometry    Entidad federativa  \
0  POLYGON ((2469550.852 1159302.643, 2469741.067...        aguascalientes   
1  MULTIPOLYGON (((1313480.513 1831458.607, 13135...       baja california   
2  MULTIPOLYGON (((1694656.345 1227647.637, 16946...   baja california sur   
3  MULTIPOLYGON (((3544897.199 946994.621, 354491...              campeche   
4  POLYGON ((2469501.165 1978862.429, 2469539.651...               chiapas   
5  MULTIPOLYGON (((

In [ ]:
merged_data

,CVEGEO,CVE_ENT,NOMGEO,geometry,Entidad federativa,cve_ent,Cantidad
0,01,1,Aguascalientes,"POLYGON ((2469550.852 1159302.643, 2469741.067...",aguascalientes,1,1425607
1,02,2,Baja California,"MULTIPOLYGON (((1313480.513 1831458.607, 13135...",baja california,2,3769020
2,03,3,Baja California Sur,"MULTIPOLYGON (((1694656.345 1227647.637, 16946...",baja california sur,3,798447
3,04,4,Campeche,"MULTIPOLYGON (((3544897.199 946994.621, 354491...",campeche,4,928363
4,05,5,Coahuila de Zaragoza,"POLYGON ((2469501.165 1978862.429, 2469539.651...",chiapas,5,5543828
5,06,6,Colima,"MULTIPOLYGON (((1157633.318 768573.525, 115772...",chihuahua,6,3741869
6,07,7,Chiapas,"POLYGON ((3558179.782 698365.65, 3558487.689 6...",ciudad de méxico,7,9209944
7,08,8,Chihuahua,"POLYGON ((1961978.327 2201082.365, 1961978.967...",coahuila de zaragoza,8,3146771
8,09,9,Ciudad de México,"POLYGON ((2802175.66 843326.718, 2801718.096 8...",colima,9,731391
9,10,10,Durango,"POLYGON ((2107050.444 1641943.398, 2109767.527...",durango,10,1832650


In [21]:
import geopandas as gpd
import altair as alt
import pandas as pd
import subprocess  # Para convertir a TopoJSON

# 1. Cargar y preparar el shapefile
#shapefile_path = "/Volumes/abdel_julio/metro/794551132173_s/mg_2024_integrado/conjunto_de_datos/00ent.shp"
shapefile_path = r'E:\\metro\\794551132173_s\\mg_2024_integrado\\conjunto_de_datos\\00ent.shp'
shapefile = gpd.read_file(shapefile_path)

shapefile["geometry"] = shapefile.geometry.simplify(tolerance=0.01)
shapefile = shapefile.set_geometry("geometry")

shapefile['CVE_ENT'] = shapefile['CVE_ENT'].astype(int)

# 2. Cargar y preparar los datos de población (df_final)
# ... (Aquí va tu código para cargar o crear df_final) ...

# 3. Filtrar datos por año
df_2020 = df_final[df_final['Año'] == 2020].copy()
df_2020['cve_ent'] = df_2020['cve_ent'].astype(int)

# 4. Agrupar datos por entidad
df_2020_agrupado = df_2020.groupby(['Entidad federativa', 'cve_ent'])['Cantidad'].sum().reset_index()

# 5. Convertir Shapefile a TopoJSON (usando ogr2ogr)
topojson_path = "mexico.topojson"  # Nombre del archivo TopoJSON de salida
subprocess.run(['ogr2ogr', '-f', 'TopoJSON', topojson_path, shapefile_path, 'estados']) # 'estados' es el nombre de la capa

# 6. Crear el mapa coroplético con Altair (usando TopoJSON)
alt.themes.enable("dark")

states = alt.topo_feature(topojson_path, 'estados') # 'estados' debe coincidir con el nombre de la capa en el comando ogr2ogr

choropleth = alt.Chart(states).mark_geoshape(
    stroke="#154360",
    strokeWidth=0.5
).transform_lookup(
    lookup='CVE_ENT',  # Clave en TopoJSON
    from_=alt.LookupData(df_2020_agrupado, 'cve_ent', ['Cantidad', 'Entidad federativa'])  # Clave y datos en df
).encode(
    color=alt.Color('Cantidad:Q', scale=alt.Scale(scheme='blues'), title="Población total en 2020"),
    tooltip=['Entidad federativa:N', 'Cantidad:Q']
).properties(
    width=1200,
    height=900
).project(
    type="conicConformal",
    center=[-102, 23],
    parallels=[14, 28],
    rotate=[97, -23],
    fit=states # Usamos states para el fit
)

choropleth.show()

FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado

In [ ]:
print(shapefile['CVE_ENT'].dtype)
print(df_2020_agrupado['cve_ent'].dtype)

int64


KeyError: 'cve_ent'

In [ ]:
df_2020_agrupado

,Entidad federativa,cve_ent,Cantidad
0,Aguascalientes,1,1425607
1,Baja California,2,3769020
2,Baja California Sur,3,798447
3,Campeche,4,928363
4,Chiapas,5,5543828
5,Chihuahua,6,3741869
6,Ciudad de México,7,9209944
7,Coahuila de Zaragoza,8,3146771
8,Colima,9,731391
9,Durango,10,1832650


In [ ]:
# # CREATE TABLE nueva_tabla AS
# SELECT
#     b.id AS id_beneficiario,
#     ig.ent_grupo_analf AS grupo_analfabetismo,
#     ig.ent_grupo_asistesc AS grupo_asistencia_escolar,
#     ig.ent_grupo_edbasinc AS grupo_educacion_basica,
#     ig.ent_grupo_sdsalud AS grupo_derechohabiencia,
#     ig.ent_grupo_ptierra AS grupo_piso_de_tierra,
#     ig.ent_grupo_hacin AS grupo_hacimiento,
#     ig.ent_grupo_nosan AS grupo_sanitario,
#     ig.ent_grupo_noagua AS grupo_agua,
#     ig.ent_grupo_nodren AS grupo_drenaje,
#     ig.ent_grupo_noelec AS grupo_electricidad,
#     rpe.pob_ent
# FROM beneficiario b
# LEFT JOIN indicadores_general ig ON b.elec_cve = ig.ncve
# LEFT JOIN relacion_pobreza_estatal rpe ON CAST(rpe.ncve AS int) = b.elec_cve AND rpe.id_periodo = 3
# WHERE ig.cve_ent = 2;

SyntaxError: invalid syntax (2232238603.py, line 1)

In [ ]:
# from sqlalchemy import create_engine
# import pandas as pd


# origen_config = {
#     "host": "montexanic.cacvh5hnp5ks.us-east-2.rds.amazonaws.com",
#     "user": "abdel",
#     "password": "R3ydb.23",
#     "database": "seguridad_historico"
# }

# destino_config = {
#     "host": "seguridadincidencias.cacvh5hnp5ks.us-east-2.rds.amazonaws.com",
#     "user": "abdel",
#     "password": "R3ydb.23",
#     "database": "Seguridad_historico"
# }


# with create_engine(
#     f"mysql+pymysql://{origen_config['user']}:{origen_config['password']}@{origen_config['host']}/{origen_config['database']}"
# ).connect() as conexion_origen, \
# create_engine(
#     f"mysql+pymysql://{destino_config['user']}:{destino_config['password']}@{destino_config['host']}/{destino_config['database']}"
# ).connect() as conexion_destino:


#     df = pd.read_sql("SELECT * FROM seguridad_historico.cat_colonias", conexion_origen)


#     schema_destino = "Seguridad_historico"
#     df.to_sql(
#         name='cat_colonias',
#         con=conexion_destino,
#         schema=schema_destino,
#         if_exists='replace',
#         index=False
#     )

# print("Migración exitosa y conexiones cerradas!")

Migración exitosa y conexiones cerradas!


In [1]:
import matplotlib.pyplot as plt
print(plt.style.available)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']
